### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import GRU

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
def italy_vigenere_cipher(string):
    return vigenere_cipher(string, 'ITALY')

letters = string.printable.split('!')[0]
encrypt = italy_vigenere_cipher
samples_count = 1000000
text_length = 10

set_characters(letters)
model_path = f"models/{encrypt.__name__}_{len(letters)}x{text_length}_best_model.h5"

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)
cipher = list(map(encrypt, text))

train_text = to_vec(text)
train_cipher = to_vec(cipher)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Bidirectional(GRU(128, activation='relu', return_sequences=True), input_shape=train_cipher.shape[1:]))
model.add(Dense(1, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 256)           99840     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 1)             257       
Total params: 100,097
Trainable params: 100,097
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=100, callbacks=callbacks)

Train on 900000 samples, validate on 100000 samples
Epoch 1/100
900000/900000 [==============================] - 799s 888us/step - loss: 31.7704 - acc: 0.4539 - val_loss: 42.2128 - val_acc: 0.2436

Epoch 00001: val_acc improved from -inf to 0.24360, saving model to models/italy_vigenere_cipher_62x10_best_model.h5
Epoch 2/100
900000/900000 [==============================] - 782s 868us/step - loss: 16.2988 - acc: 0.5811 - val_loss: 6.2746 - val_acc: 0.7302

Epoch 00002: val_acc improved from 0.24360 to 0.73021, saving model to models/italy_vigenere_cipher_62x10_best_model.h5
Epoch 3/100
900000/900000 [==============================] - 785s 872us/step - loss: 13.3163 - acc: 0.6096 - val_loss: 9.7335 - val_acc: 0.6553

Epoch 00003: val_acc did not improve from 0.73021
Epoch 4/100
900000/900000 [==============================] - 787s 874us/step - loss: 11.0175 - acc: 0.6329 - val_loss: 2.2226 - val_acc: 0.8644

Epoch 00004: val_acc improved from 0.73021 to 0.86442, saving model to models/it

### Predicting and Evaluating the Model

In [6]:
model = load_model(model_path)

In [7]:
test_cyhper = encrypt('HelloWorld')
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: HelloWorld
Test cipher vs encrypted prediction: p7V6mEh16b vs p7V6mEh16b
